Com o Resultado do apriori, descobrimos que a carta jirachi uma carta amplamante utilizadas entre os jogadores nos torneios de 2019.
Iremos utilizar um algoritmo SD como alvo o pokemon jirachi para encontrar combinacoes de cartas que influenciem um jogador chegar subir no ranque em um torneio

## Bibliotecas utilizadas e carregando dataset

In [140]:
CONSIDERAR_TREINADORES = False

import numpy as np
import pandas as pd
import pysubgroup as ps

df_original = pd.read_csv('./Data/tournaments.csv')
df = df_original.copy()


## Eliminar cartas treinador se quiser

In [141]:
if not CONSIDERAR_TREINADORES:
    df = df[df['type_card'] != 'Trainer']

## Filtragem de colunas no dataset e limpeza de linhas com coluna torneio vazias

In [142]:
columns_to_remove = [
    'id_card', 'amount_card', 'price_card', 'combo_type_id', 'name_tournament',
    'combo_type_name', 'name_player', 'category_tournament', 'type_card',
    'month_tournament', 'day_tournament', 'valid_rotation_at_tournament', 
    'rotation_name', 'year_begin', 'month_begin', 'day_begin', 'all_time_score', 'country_tournament', 'country_player',
]
df.drop(columns=columns_to_remove, inplace=True)
df = df[df['region_tournament'].notna()]

df.head(1)

,name_card,energy_type_card,id_player,ranking_player_tournament,id_tournament,region_tournament,year_tournament
7462,Charmander,Fire,649,1,385,SA,2023


## Adaptando dados para pordermos usar SD

Como somente `ranking_player` do tipo numerico, vamos transformalo em binario em que menor que 15 seria 1, estamos buscando cartas que aumentem a probabilidade de um jogador subir de rank acima de 15

In [143]:
# todos os ranking players acima de 15 sao 1
df['ranking_player_tournament'] = df['ranking_player_tournament'].apply(lambda x: 1 if x < 15 else 0)

df.head(1)

,name_card,energy_type_card,id_player,ranking_player_tournament,id_tournament,region_tournament,year_tournament
7462,Charmander,Fire,649,1,385,SA,2023


## Separando por regiao e verificando a qualidade dos subgrupos de cartas que aumentem a probabilidade de um jogador subir no rank

In [144]:
def agrupa_deck_rank(df_region):
    #filtra torneios da regiao
    torneios = df_region['id_tournament'].unique()

    lista_decks_rank = []
    for torneio in torneios:
        df_torneio = df_region[df_region['id_tournament'] == torneio]
        jogadores = df_torneio['id_player'].unique()
        for jogador in jogadores:
            df_jogador = df_torneio[df_torneio['id_player'] == jogador]
            cartas_jogador_rank = df_jogador['name_card'].unique().tolist()
            cartas_jogador_rank.append(df_jogador['ranking_player_tournament'].iloc[0])
            lista_decks_rank.append(cartas_jogador_rank)
    return lista_decks_rank

def subgrupo_dataset_region(df_region):
    lista_decks_rank = agrupa_deck_rank(df_region)
    cartas = df_region['name_card'].unique()
    data = []
    for deck_rank in lista_decks_rank:
        pokemons_no_deck_rank = {carta: (carta in deck_rank[:len(deck_rank)-1]) for carta in cartas}
        pokemons_no_deck_rank['ranking_player_tournament'] = deck_rank[-1]
        data.append(pokemons_no_deck_rank)
    # Converte a lista de dicionários em um DataFrame
    df_decks_cartas = pd.DataFrame(data)

    return df_decks_cartas

def calcula_subgrupo(df):
    target = ps.BinaryTarget('ranking_player_tournament',1)
    search_space = ps.create_selectors(df, ignore=['ranking_player_tournament'])
    task = ps.SubgroupDiscoveryTask(
        df,
        target, 
        search_space, 
        result_set_size=10, 
        depth=5, 
        qf=ps.WRAccQF()  # Quality function to evaluate subgroups
    )
    result = ps.BeamSearch().execute(task)
    return result

## Por regiao

In [145]:
regions = df['region_tournament'].unique()
for region in regions:
    print(region)
    df_region = df[(df['region_tournament'] == region) & (df['year_tournament'] == 2019)]
    dataset = subgrupo_dataset_region(df_region)
    result = calcula_subgrupo(dataset)
    # Display the results
    for row in result.to_dataframe().itertuples():
        print(row.subgroup)
    print(result.to_dataframe())

SA
Heatran-GX==False AND Hoopa==False AND Keldeo-GX==False AND Latios-GX==False AND Xerneas-GX==False
Heatran-GX==False AND Hoopa==False AND Keldeo-GX==False AND Latios-GX==False AND Tyrogue==False
Florges==False AND Heatran-GX==False AND Hoopa==False AND Keldeo-GX==False AND Latios-GX==False
Flabebe==False AND Heatran-GX==False AND Hoopa==False AND Keldeo-GX==False AND Latios-GX==False
Heatran-GX==False AND Hoopa==False AND Keldeo-GX==False AND Latios-GX==False AND Oranguru==False
Heatran-GX==False AND Hoopa==False AND Keldeo-GX==False AND Latios-GX==False AND Zeraora-GX==False
Heatran-GX==False AND Hoopa==False AND Keldeo-GX==False AND Latios-GX==False AND Wobbuffet==False
Heatran-GX==False AND Hoopa==False AND Keldeo-GX==False AND Latios-GX==False AND Volcanion==False
Heatran-GX==False AND Hoopa==False AND Keldeo-GX==False AND Latios-GX==False AND Tapu Koko ♢==False
Heatran-GX==False AND Hoopa==False AND Keldeo-GX==False AND Latios-GX==False AND Sawsbuck==False
    quality          

## Ignorando Regioes

In [146]:
df_all = df[(df['year_tournament'] == 2019)]
dataset = subgrupo_dataset_region(df_all)
result = calcula_subgrupo(dataset)
for row in result.to_dataframe().itertuples():
    print(row.subgroup)
print(result.to_dataframe())

Cryogonal==False AND Marshadow-GX==False AND Poipole==False AND Tapu Koko-GX==False AND Weavile==False
Cryogonal==False AND Marshadow-GX==False AND Tapu Koko-GX==False AND Weavile==False AND Wobbuffet==False
Cryogonal==False AND Magcargo-GX==False AND Marshadow-GX==False AND Tapu Koko-GX==False AND Weavile==False
Cryogonal==False AND Eevee==False AND Marshadow-GX==False AND Tapu Koko-GX==False AND Weavile==False
Cryogonal==False AND Marshadow-GX==False AND Poipole==False AND Tapu Koko-GX==False AND Wobbuffet==False
Cryogonal==False AND Marshadow-GX==False AND Poipole==False AND Sneasel==False AND Tapu Koko-GX==False
Cryogonal==False AND Marshadow-GX==False AND Naganadel==False AND Tapu Koko-GX==False AND Weavile==False
Cryogonal==False AND Marshadow-GX==False AND Tapu Koko-GX==False AND Weavile==False AND Wooper==False
Cryogonal==False AND Marshadow-GX==False AND Quagsire==False AND Tapu Koko-GX==False AND Weavile==False
Cryogonal==False AND Magcargo-GX==False AND Marshadow-GX==False A